#MA705 Yelp Report - Multipe Variable Summaries (spring 2019, codingwatchcity)

## Introduction
So far, we did table joining and data cleaning work in notebook **1.View**, and then explored all individual variables, getting a good understanding about the basic statistic information of them. 

After that we decide to further analyze the relationships between those variables, aiming at discovering the factors related to the star ratings for Starbucks. 

Multiple variables studied are listed below:
1. Averaged `stars`(average rating for each Starbucks store (from 1 to 5 steped by 0.5) group by states) and `states`(names of 11 states). see cmd 5
2. `review_stars`(rating from each review (from 1 to 5 steped by 1)) and `stars`(ratings for each business). see cmd 9
3. `review_count`(the number of review of each Starbucks.) and `review_stars`(rating from each review (from 1 to 5 steped by 1)). see cmd 28
4. `stars`(rating for each Starbucks store (from 1 to 5 steped by 0.5)) and `states`(names of 11 states). see cmd 41
5. `review_count`(the number of review of each Starbucks.) and `states`(names of 11 states). see cmd 67
6. `hours`(the opening hours of Starbucks on Saturday and Suncday.) and `states`(names of 11 states). see cmd 5 in notebook 3a
7. `hours`(the opening hours of Starbucks on Saturday and Suncday.) and `stars`(rating for each Starbucks store (from 1 to 5 steped by 0.5)). see cmd 28 in notebook 3a
8. `stars`(rating for each Starbucks store (from 1 to 5 steped by 0.5)), `hours`(the opening hours of Starbucks on Saturday and Suncday.) and `states`(names of 11 states). see cmd 40 in notebook 3a
9. `attribute`(park_info:garage,street and lot), averaged `stars`(rating for each Starbucks store (from 1 to 5 steped by 0.5)) and `states`(names of 11 states). see cmd 44 in notebook 3a

For each of the variable pairs, we check its statistical features(i.e. correlation coefficient table). We also create visualizations, such as boxplots and histograms, for some of the variables. Thos analysis enhances the understanding of the dataset.

In [3]:
%python
import pandas            as pd
import numpy             as np
import seaborn           as sns
import matplotlib.pyplot as plt
sns.__version__

Out[ 50 ]: '0.9.0'

In [4]:
%run "./1. Views (MA705, Yelp)"

##1. One sample t-test

After examining the star ratings for Starbucks as a single variable, we want to check if average star ratings of some states are very different from the population average star rating. We will perform one sample t-test for each state, calculate the corresponding p-value, and then check if the average star rating deviates from the population mean on a alpha level equals 0.05.   

Firstly, we use `select` statement select city, std(stars), avg(stars), state, count(*) from `yelp_starbucks`, and group results by state; then the `toPandas` method transforms it into Pandas DataFrame `sta`. Having the output, we retrieve sample mean, population mean, sample standard deviation and population size from the sta and then store the results in 4 Numpy arrays. After that, t values are calculated and put in a new pandas dataframe. The `concat` method concats three DataFrames(including an empty dataframe which will be used to store p value later) together, resulting a dataframe with 11 rows and 6 columns. We noticed that some of the t-values are negative, whereas the others are positive. Thus, we develop a if else statement to distinguish the different t values, and use `cdf` method in stats package to calculate p values respectively for each state. The result is a DataFrame containing the names of 11 states and corresponding star rating standard deviation, average star rating, sample size, t-values and p-values.

In [7]:
 %python
from scipy import stats
sta=spark.sql('select std(stars),avg(stars),state,count(*) from business_new group by state').toPandas()
# the average star of each state
u1 = np.array(sta.iloc[:,[1]])
# the number of review of each state
n=np.array(np.array(sta.iloc[:,[3]]))
# u0=average of population,sig=standard deviation of population
u0=(spark.sql('select avg(stars) from business_new').toPandas())
sig=np.array(np.array(sta.iloc[:,[0]]))
# rep u0 for 11 times
newu0 = np.array(pd.DataFrame(np.repeat(u0.values,11,axis=0)))
empty = np.array(pd.DataFrame(np.repeat(0,11,axis=0)))

#calculate t(array),td(dataframe)
t=(u1-newu0)/(sig/(n**0.5))
tvalue=pd.DataFrame(t,columns=['t-value'])
pvalue=pd.DataFrame(empty,columns=['p-value'])
result=pd.concat([sta,tvalue,pvalue],axis=1);
for i in range(11):
  if result.iloc[i,4] > 0 :
    result.iloc[i,5]=round(2*(1-stats.t.cdf(result.iloc[i,4],df=result.iloc[i,3])),4)
  else:
    result.iloc[i,5]=round(2*stats.t.cdf(result.iloc[i,4],df=result.iloc[i,3]),4)
print(result)

stddev_samp(stars) avg(stars) state count(1) t-value p-value
0 0.659582 3.109375 AZ 256 -3.702249 0.0003
1 0.353553 3.250000 SC 2 -0.047985 0.9661
2 0.681607 3.522727 QC 44 2.537379 0.0148
3 0.645919 3.018817 NV 186 -5.134573 0.0000
4 0.795168 3.281250 WI 16 0.096854 0.9240
5 0.716716 3.141791 NC 67 -1.372819 0.1744
6 0.632456 3.000000 IL 6 -1.014707 0.3494
7 0.650027 3.500000 OH 72 3.106839 0.0027
8 0.674569 3.482456 PA 57 2.467406 0.0166
9 0.602677 3.441406 ON 256 4.763015 0.0000
10 0.658891 3.343750 AB 80 1.109786 0.2704

From the results, we found that State NC, AB, SC, WI, and IL have insignificant p-value; further, we noticed that SC, WI and IL have sample numbers below 30, which means that the t-test may not be valid for those three states, reducing the "abnormal" states to AB and NC. We decide to do some further research studying why their average star ratings deviate from the population average.

##2.`stars` in both `business` and `review` table

Since we have build a combined view and a dataframe for `business` table and `review` table, which are `starbucks` view and `starb_df`, we can retrieve `stars` information directly.

**(1) Boxplots for `stars` and `review_stars`**

By using `.catplot` funtion in seaborn package, we can visually compare the rough ranges and critical percentiles of `stars` and `review_stars`.

In [13]:
%python
sns.catplot(data=starb_df[["stars","review_stars"]], kind="box",
            height=4, width=0.2,aspect=6/4);
display(plt.show())

From the output above, we can see either `stars` in `business` table or `stars` in `review` table ranging from 1 to 5. Both of the medians are about 3. However, the normal range of `stars` in `business` table is much shorter than `stars` in `review` table. In `business` table, the 25th pecentile is about 2.5, the 75th pecentile is about 3.5. In contrast, in `review` table, these two numbers are about 2.0 and 4.0. We can also conclude `review_stars` has a higher standard deviation than that of `stars`.

**(2) Compare `stars` of each store with its average `review_stars`**

We want to check the relation between star rating and the average star rating of reviews of each store, trying to find out if there is any correlation between `stars` and `average review stars`. It is highly possible that these two variables have strong correlationship.

In [17]:
%sql
select business_id, stars,avg(review_stars) as avg_review_star from starbucks
group by business_id,stars
order by business_id

business_id,stars,avg_review_star
-9Ir5OiFiOszIgeFgalXcw,3.0,3.1794871794871793
-BJ0Z28LoETB_ZsdA4Ikeg,3.0,3.0
-Mu6qCYjrcvBgwIhLoh08g,3.5,3.6
-P4Mmh-0pkC4yb6cyPJhbA,3.0,3.1666666666666665
-RxkfrFaOLpE6Be3zGfsVg,4.0,4.2
-TGwtTiieh_JLml5_VRMQA,3.5,3.5
-aoMfFDMmJOgkOT3dacwpg,3.0,2.909090909090909
-i3dOjumvOw-52aGXU1xDg,3.5,3.3095238095238093
-ifVnqwrgCE3Nc4svkJDMg,2.0,2.0377358490566038
-yD0mm71HtZZDE9meI9FqA,3.5,3.5


From the output, we can compare the `stars` and `avg_review_star` of each Starbucks. It shows the `stars` has a very strong relationship with `avg_review_star`.

We transform the sql table above into a pandas dataframe. 

We first use the `select` statment to select `final_stars` and `avg_review_star` from `bus_review_starbuck`. The `spark.sql` turns it into a spark dataframe and the `toPandas()` turns it into pandas dataframe.

In [20]:
%python
starb_stars_df = spark.sql ('select business_id, stars as final_stars,avg(review_stars) as avg_review_star from starbucks group by business_id,stars').toPandas()

The output above is a datafram named `starb_stars_df` containing `final_stars` and `avg_review_star` from `starbucks` view. We will use this dataframe to draw a graph.

We want to draw scatterplot to check the relationship between `stars` and `avg_review_star.`

The `plt.clf()` function clears entire current figure, and `sns.scatterplot()` creates a plot with data from `stars_starbuck`, the data on x-axis is `final_stars`, and data on y-axis is `avg_review_star`. The `display()` and `plt.how()` function displays the plot.

In [23]:
%python
plt.clf()
sns.scatterplot(data=starb_stars_df, x='avg_review_star',y='final_stars')
display(plt.show())

The output shows the relationship between the star rating and the average star rating given by reviewers of a Starbucks store. We can see that it has a strong linear relationship. The final star rating will increase by the increasing of the average star rating given by reviewers. Thus, the star rating given by reviewers has a big impact of the final star rating of a starbuck.

We want to check the correlation between `stars` and `avg_review_star`.

In [26]:
%python

starb_stars_df.corr()

Out[ 64 ]: 
 final_stars avg_review_star
final_stars 1.000000 0.977428
avg_review_star 0.977428 1.000000

By using `corr()`method, we can see the correlation is 0.977428, which shows a very strong correlation just as we observed above.

##3. `stars` in `business` table and `review_id` in `review` table

We want to check the relation between the `star` in `business` table and the number of `review_id` in `review` table. We want to check if a store with a high number of reviews will achieve a higher overall star ratings.   

Sine we have combined the two original tables into `starbucks` view, we just use the `starbucks` view to retrieve information needed.

**(1) Create a dataframe with the information of `stars` and `review_id` numbers**

We create sql table to retrieve `stars` and `review_id` information.

We first use the `select` statment to select `business_id`,`count(review_id)` and `stars` from `starbucks` to gain a basic understanding of the relationship between stars and count(review_id). It's hard to find the relationship in the table listed below. So we will try to perform more anlysis.

In [32]:
%sql
select business_id, stars, count(review_id) as review_count 
from starbucks
group by business_id, stars

business_id,stars,review_count
-TGwtTiieh_JLml5_VRMQA,3.5,8
6nCcjMGRfpIiIi5JMgUNrg,3.0,8
8zhQrSmdFFowL5fsGFPdpg,3.5,12
X26lmoKvv_KvW6WcAvWvrA,3.5,10
h8eDSoKHpp-4l5G5_JPtFA,3.5,26
BCRGQtUtp_ICaRdQNcPEPw,3.5,6
o2811xZ0ufRi7WcsqWiGzw,4.0,3
5KRe7GMvbr-0pyncIM48-A,2.5,6
6umdBLNv5DFCoGxjnH6Ajg,4.0,7
7v-2HkyrfYkPf471v4gCmg,2.5,35


From the output above, we can get the information of `business_id` and `stars` columns. Meanwhile, we count the number of `review_id`.

We tranform the sql table above into pandas dataframe.

We use the `select` statment to select `business_id`, `stars`, `count(review_id)` data from `starbucks`. `spark.sql` turns it into spark dataframe, and `toPandas()` turns it into a pandas dataframe, which can be used in plotting the graph in the next step.

In [35]:
%python
starb_revcount_df = spark.sql('select business_id, stars, count(review_id) as review_count from starbucks group by business_id, stars').toPandas()

The output above is a datafram named `starb_revcont_df ` containing `business_id`,`stars` and `review_count` from `starbucks` view. We will use this dataframe to draw a graph.

**(2) Visualize the relation between `stars` and `review_count`**

We want to draw scatterplot to check the relationship between `stars` and `review_count`.   

The `plt.clf()` function is used at the beginning to clear current figure. We then use `sns.scatterplot()` to create a plot with data from `starb_revcount_df`. The data on x-axis is `review_count` and on y-axis is `stars`. At the end, the `display()` function is used displays the plot.

In [39]:
%python
plt.clf()
sns.scatterplot(data=starb_revcount_df, y='stars',x='review_count')
display(plt.show())

The plot shows the relationship between the final star rating and the number of review of a Starbucks store. We can see that a store with more number of reviews, the final star rating is more likely to be in the range of 2-4. The star rating is normal distributed with the number of review. It turns out that if a stores with very low or very high star rating, it is highly possible that the number of review is very low.

##4. `stars` and `state` in `business` table

In the previous analysis, we found that "NC" and "AB" has large p_value compare to the population. We would like to look into these states in this part and find out reasons.

**(1) Check the average `stars` by `state`**

We want to retrieve the information of ` state` and `avg(stars)`.

In [45]:
%sql
select state, avg(stars) from business_new
group by state
order by avg(stars)desc;

state,avg(stars)
QC,3.522727272727273
OH,3.5
PA,3.482456140350877
ON,3.44140625
AB,3.34375
WI,3.28125
SC,3.25
NC,3.1417910447761193
AZ,3.109375
NV,3.0188172043010755


The table contains the information of `state` and `stars`. The result is grouped by `state` and order by average stars, in descending order. From the output, we can conclude that the top three states in average star rating are **QC**, **OH**, **PA** and the bottom three states are **IL**, **NV**, **AZ**. Comparing the average star of each state, **NC** and **AB** are in the 5th and 8th position. We don't think the average star will be the reason sperating them from population. Then we will try to look into the star distribution of these states to find out reasons.

**(2) Check the distribution of `stars` in **NC** state**

We already had the graph of distribution of stars in all states when we explored the single variables. The graph shows that it is normally distributed. We want to check the distribution of stars in NC and AB states to see if it is also a normal distribution.

In [49]:
%sql
drop view if exists NC_star;
create temporary view NC_star as 
select count(stars),stars from business_new
where state == 'NC'
group by stars;

The view we created above is called `NC_star`. We use it to store count number of stars for each star rating in the state NC.

We tranform the sql view `NC_star` above into pandas dataframe.

In [52]:
%python
stateNC_stars_df = spark.sql ('select * from NC_star ').toPandas()


The output above is a datafram named `stateNC_stars_df ` containing `stars` and `count(stars)`. We will use this dataframe to draw a graph.

We want to draw barcharts to check the relationship between `stars` and `count(stars)`.    

`plt.clf()` function clears entire current figure. `sns.scatterplot()` is used to create a plot using data from `stateNC_stars_df`, with x-axis is `stars` and y-axis is `count(stars)`, and the `display()` function displays the plot at the end.

In [55]:
%python
plt.clf()
sns.barplot(data=stateNC_stars_df, y='count(stars)',x='stars')
display(plt.show())

The plot shows the relationship between the stars rating and the number of stores having this rating in the **NC** state. We can see that most of stores has star rating from 2.5 to 4.0, which is a normal distribution. From this chart, the distribution of stars rating in each state is not the reason seperating the **NC** state from other states.

**(3) Check the distribution of `stars` in **AB** state**

We created a view called `AB_star` to store count number of stars for each star rating in the state AB.

In [59]:
%sql
drop view if exists AB_star;
create temporary view AB_star as 
select count(stars),stars from business_new
where state == 'AB'
group by stars;

The view we created above is called `AB_star`. We use it to store count number of stars for each star rating in the state **AB**.

We transform the sql view `AB_star` above into a pandas dataframe.

The `spark.sql` turns data selected by `select` statement into spark dataframe, and `toPandas()` turns it into a pandas dataframe, which can be used in plotting the graph in the next step.

In [62]:
%python
stateAB_stars_df = spark.sql ('select * from AB_star ').toPandas()


The output above is a dataframe named `stateAB_stars_df` containing `stars` and `count(stars)`. We will use this dataframe to draw a graph.

We want to draw a barchart to check the relationship between `stars` and `count(stars)`.

In [65]:
%python
plt.clf()
sns.barplot(data=stateAB_stars_df, y='count(stars)',x='stars')
display(plt.show())

The plot shows the relationship between the stars rating and the number of stores having this rating in the AB state. We can see that most of stores have star rating from 2.5 to 4.0, which is also a normal distribution. From this chart, the distribution of stars rating in each state is not the reason seperating the AB state from other states.

##5. `review_count` and `state`

After checking the star rating distribution of each state, we are trying to find if the `review_count` is the reason that causes two states seperating from the population.

We want to create sql view to retrieve the information of `state` and `review_count`.

In [70]:
%sql
drop view if exists state_reviewcount;
create temporary view state_reviewcount as 
select state, sum(review_count)
from business_new
group by state;


The output is a sql view named `state_reviewcount`, which stores information of  `state` and `sum(review_count)`.

We tranform the sql view `state_reviewcount` above into a pandas dataframe.  

The `spark.sql` turns data selected by `select` statement into spark dataframe, and `toPandas()` turns it into a pandas dataframe. The dataframe could be used for plotting.

In [73]:
%python
state_reviewcount_df = spark.sql ('select * from state_reviewcount ').toPandas()

We want to draw a barchart to check the relationship between `review_count` and `state`.   

`plt.clf()` function clears the current figure. The `sns.scatterplot()` is used to create a plot using data from `stateAB_stars_df`, with x-axis is `state` and y-axis is `count(stars)`, and the `display()` function displays the plot at the end.

In [75]:
%python
plt.clf()
sns.barplot(data=state_reviewcount_df, y='sum(review_count)',x='state')
display(plt.show())

The output is a bachart graph. It shows the relationship between the states and the total number of reviewin each state. Each bar represents the number of review. The higher the bar is, the larger the number of review a state has. We can see that **NC** and **AB** states have relatively small number of review. As we discussed above, the `review_count` can affect star rating. It could be the reason that seperating **NC ** and **AB** from population.